# Лабораторная работа №4

> Разработка и исследование нечёткого регулятора

* Научиться представлять атрибуты знаний;
* Разработать алгоритмы создания и редактирования компонентов баз знаний;
* Разрабатывать интерфейсы пользователей для редактора правил;
* Разработать форматы представления атрибутов и правил в информационных системах.

In [440]:
# Импорт необходимых библиотек
import numpy as np
import skfuzzy as fuzz
import yaml
import matplotlib.pyplot as plt

# Предел после которого считаем, что значение принадлежит множеству
fuzzy_limit = 0.25

# Анализируемые пользователи
users = [
    {'age': 20, 'income': 20000, 'credit_history': 0},
    {'age': 15, 'income': 25000, 'credit_history': 1},
    {'age': 35, 'income': 80000, 'credit_history': 1},
    {'age': 22, 'income': 30000, 'credit_history': 1},
    {'age': 80, 'income': 100000, 'credit_history': 1},
    {'age': 50, 'income': 80000, 'credit_history': 0},
    {'age': 30, 'income': 40000, 'credit_history': 1},
    {'age': 25, 'income': 60000, 'credit_history': 1},
    {'age': 45, 'income': 70000, 'credit_history': 0},
    {'age': 40, 'income': 55000, 'credit_history': 0.5},
    {'age': 22, 'income': 45000, 'credit_history': 0.5}
]

In [441]:
# Загрузка базы знаний из файла
with open('knowledge_base.yml', 'r') as file:
    knowledge_base = yaml.load(file, Loader=yaml.FullLoader)

In [442]:
# Функция подсчитывает степень принадлежности значения к каждому нечеткому множеству
def fuzzify(value, fuzzy_sets):
    memberships = {}
    for fuzzy_set_name, points in fuzzy_sets.items():
        x = np.linspace(0, max(points), 100)
        membership_function = fuzz.trapmf(x, points)
        memberships[fuzzy_set_name] = fuzz.interp_membership(x, membership_function, value)
    return memberships

In [443]:
# Функция оценивает условие для каждого подмножества антецедентов (предшествующих условий)
def eval_condition(attributes, condition, knowledge_base):
    output = []
    for sub_condition in condition:
        attr_name = sub_condition["attribute"]
        if attr_name in attributes:
            attr_value = attributes[attr_name]
            fuzzy_set = sub_condition.get("fuzzy_set")
            if fuzzy_set:
                output.append(
                    fuzzify(attr_value, knowledge_base["attributes"][attr_name]["fuzzy_sets"])[fuzzy_set] > 0.5)
            else:
                output.append(attr_value == sub_condition["value"])
        else:
            return False
    return all(output)

In [444]:
# Функция для визуализации
def visualize_fuzzy_sets(user, knowledge_base):
    for attr_name, attr_value in user.items():
        if attr_name in knowledge_base["attributes"]:
            attribute = knowledge_base["attributes"][attr_name]
            if "fuzzy_sets" in attribute:
                for fuzzy_set_name, points in attribute["fuzzy_sets"].items():
                    x = np.linspace(0, max(points), 100)
                    membership_function = fuzz.trapmf(x, points)
                    plt.plot(x, membership_function, label=fuzzy_set_name)

                memberships = fuzzify(user[attr_name], attribute["fuzzy_sets"])

                # Вычисление максимального уровня членства
                max_membership = max(memberships.values())

                for fuzzy_set_name, membership in memberships.items():
                    # Проверка, если степень принадлежности равна максимальной, то рисуем точку
                    if membership == max_membership:
                        plt.scatter(
                            user[attr_name],
                            membership,
                            color='yellow',
                            s=100,
                            label=f"user"
                        )
                        break

                plt.title(attr_name)
                plt.ylabel("Membership")
                plt.xlabel("Value")
                plt.legend(loc="center right")
                plt.grid(True)
                plt.show()

In [445]:
# Функция применяет правила из базы знаний к атрибутам пользователя
def apply_rules(attributes, rules, knowledge_base):
    for rule in rules:
        if eval_condition(attributes, rule["antecedent"], knowledge_base):
            decision = rule["consequent"]["value"]
            print(f"Для пользователя с атрибутами {attributes}, решение: {decision}")
            return rule["consequent"]
    decision = "rejected"
    print(f"Для пользователя с атрибутами {attributes}, решение: {decision}")
    return {"attribute": "credit", "value": decision}

In [446]:
# Для каждого пользователя рисуем график
for user in users:
    apply_rules(user, knowledge_base['rules'], knowledge_base)
    #visualize_fuzzy_sets(user, knowledge_base)

Для пользователя с атрибутами {'age': 20, 'income': 20000, 'credit_history': 0}, решение: rejected
Для пользователя с атрибутами {'age': 15, 'income': 25000, 'credit_history': 1}, решение: rejected
Для пользователя с атрибутами {'age': 35, 'income': 80000, 'credit_history': 1}, решение: approved
Для пользователя с атрибутами {'age': 22, 'income': 30000, 'credit_history': 1}, решение: rejected
Для пользователя с атрибутами {'age': 80, 'income': 100000, 'credit_history': 1}, решение: rejected
Для пользователя с атрибутами {'age': 50, 'income': 80000, 'credit_history': 0}, решение: rejected
Для пользователя с атрибутами {'age': 30, 'income': 40000, 'credit_history': 1}, решение: approved
Для пользователя с атрибутами {'age': 25, 'income': 60000, 'credit_history': 1}, решение: approved
Для пользователя с атрибутами {'age': 45, 'income': 70000, 'credit_history': 0}, решение: rejected
Для пользователя с атрибутами {'age': 40, 'income': 55000, 'credit_history': 0.5}, решение: approved
Для пол